In [142]:
import os, sys
from tqdm import tqdm
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))

import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt

from src.features import intended_receiver, extract_player_pos, extract_pass
from src.visualization import plot_action
from src.preprocess_data import make_freeze_frame
import math
from src.labels import get_intended_receiver, get_nearest_receiver

# this is very useful as it makes sure that always all columns and rows of a data frame are displayed
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [146]:
import handcalcs.render

In [154]:
np.finfo(float).eps

2.220446049250313e-16

In [153]:
%%render
        a = ball_after - ball_coo                      # 0.4초 ball 위치 - 0.04초 볼 위치
        b = receiver_coo - ball_coo 

angle = np.arccos(np.clip(np.sum(a * b, axis=1) / (np.linalg.norm(a) * np.linalg.norm(b, axis=1) + np.finfo(float).eps), -1, 1))

TypeError: Field elements must be 2- or 3-tuples, got '3'

<h2> Data Load </h2>

In [2]:
match1 = pd.read_csv('../metrica-data/preprocess-data/tracking-data/match1.csv')
match2 = pd.read_csv('../metrica-data/preprocess-data/tracking-data/match2.csv')
match3 = pd.read_csv('../metrica-data/preprocess-data/tracking-data/match3.csv')
all_events = pd.read_csv('../metrica-data/EPV-data/labelling-all-match.csv')
all_events['freeze_frame'] = all_events['freeze_frame'].apply(ast.literal_eval)
all_events['Intended_Receiver'] = all_events['Baseline Intended-Receiver'].apply(ast.literal_eval)

In [3]:
all_events[all_events['event_id'] == 340]

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,freeze_frame,accurate,value_label,Baseline Intended-Receiver,game_id,event_id,True Intended-receiver,no pass,Intended_Receiver
340,Away,BALL LOST,INTERCEPTION,1,24433,977.32,24486,979.44,B18,NaN,12.96,12.24,9.72,7.92,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'B23', 'end_x': 21.5136, 'end_...",1,340,NaN,1.0,"{'dist': {'ID': 'B23', 'end_x': 21.5136, 'end_..."


In [100]:
len(all_events[(all_events['accurate'] == 1)&
           (all_events['eventName'] == 'Pass')&
           (all_events['start_frame']< all_events['end_frame'])&
           (all_events['game_id'] == 2)])

964

In [101]:
all_events.head()

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,freeze_frame,accurate,value_label,Baseline Intended-Receiver,game_id,event_id,True Intended-receiver,no pass,Intended_Receiver
0,Away,SET PIECE,KICK OFF,1,1,0.04,0,0.00,B19,NaN,NaN,NaN,NaN,NaN,1,0,0,2,SET PIECE,{},NaN,0,{},1,0,NaN,NaN,{}
1,Away,PASS,PASS,1,1,0.04,3,0.12,B19,B21,48.6,28.08,59.40,30.96,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B21', 'end_x': 59.66244, 'end...",1,1,NaN,NaN,"{'dist': {'ID': 'B21', 'end_x': 59.66244, 'end..."
2,Away,PASS,PASS,1,3,0.12,17,0.68,B21,B15,59.4,30.96,62.64,15.12,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B15', 'end_x': 63.06444, 'end...",1,2,NaN,NaN,"{'dist': {'ID': 'B15', 'end_x': 63.06444, 'end..."
3,Away,PASS,PASS,1,45,1.80,61,2.44,B15,B19,59.4,13.68,48.60,22.32,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B19', 'end_x': 48.33756, 'end...",1,3,NaN,NaN,"{'dist': {'ID': 'B19', 'end_x': 48.33756, 'end..."
4,Away,PASS,PASS,1,77,3.08,96,3.84,B19,B21,48.6,23.04,52.92,33.84,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B21', 'end_x': 52.67376, 'end...",1,4,NaN,NaN,"{'dist': {'ID': 'B21', 'end_x': 52.67376, 'end..."


In [111]:
def get_nearest_receiver(events, df_track):
    events['Intended_Receiver'] = [{} for _ in range(len(events))]
    
    # angle : receiver - 0.04초 볼 위치 - 0.4초 볼 위치 각도의  중 최솟값 
    ball_receiver_angle = []

    for idx, action in events.iterrows():
        Intended_dict = {}
        
        # after_frame: 0.4까지 데이터만 있으므로, 현재 frame에서 9(3.6초)frame 뒤로 잡음.
        # 만약 0.4초 안에 끝날 경우, Intended Receiver를 예측하지 않음.
        after_frame = action['start_frame'] + 9
        if action['end_frame'] < after_frame:
             events.at[idx,'Intended_Receiver'] = 0
             continue
        
        # tracking data 불러오기
        track = df_track[df_track['frame'] == action['start_frame']]
        track_after = df_track[df_track['frame'] == after_frame]
        passer = action["from"]
        team = passer[0]

        # ball 위치
        ball_coo = np.array([track['ball_x'].iloc[0], track['ball_y'].iloc[0]]).reshape(-1, 2)
        ball_after = np.array([track_after['ball_x'].iloc[0], track_after['ball_y'].iloc[0]]).reshape(-1, 2)

        # receiver 위치
        feature_types = ["_x", "_y"]
        player_names = [c[:3] for c in track.filter(regex=f'_x').iloc[0].index if c[0] == team and c[:3] != passer]
        teammate_cols = [f'{c}{pos}' for c in player_names if c[0] == team for pos in feature_types]
        teammate_input = track[teammate_cols].dropna(axis=1)
        player_names = [c[:3] for c in teammate_input.filter(regex=f'_x').iloc[0].index]
        receiver_coo = np.array(list(teammate_input.values)).reshape(-1,2)
        
        #각 팀원 선수의 인덱스 -> 선수ID를 매핑하는 딕셔너리 : receiver ID찾는 작업
        player_index_dict = {index:player for index, player in enumerate(player_names)}

        a = ball_after - ball_coo                      # 0.4초 ball 위치 - 0.04초 볼 위치
        b = receiver_coo - ball_coo                    # receiver 위치 - 0.04초 볼 위치

        # dist : 시작 지점 공의 위치와 receiver 거리
        dist = np.sqrt((receiver_coo[:, 0] - ball_coo[:, 0]) ** 2+ (receiver_coo[:, 1] - ball_coo[:, 1]) ** 2)
        exp_receiver_dist = np.argmax((np.amin(dist) / dist))
        Intended_dict['dist_nearest'] = {'ID':player_index_dict[exp_receiver_dist], 
                                 'end_x':receiver_coo[exp_receiver_dist][0], 'end_y':receiver_coo[exp_receiver_dist][1]}

        # angle : a, b 사이의 각도
        angle = np.arccos(np.clip(np.sum(a * b, axis=1) / (np.linalg.norm(a) * np.linalg.norm(b, axis=1) + np.finfo(float).eps), -1, 1))
        exp_receiver_angle = np.argmax(np.amin(angle) / angle)
        Intended_dict['angle_nearest'] = {'ID':player_index_dict[exp_receiver_angle], 
                                 'end_x':receiver_coo[exp_receiver_angle][0], 'end_y':receiver_coo[exp_receiver_angle][1]}
        
        # dist and angle
        exp_receiver_dist_and_angle = np.argmax((np.amin(dist) / dist) * (np.amin(angle) / angle))
        Intended_dict['dist and angle'] = {'ID':player_index_dict[exp_receiver_dist_and_angle], 
                                           'end_x':receiver_coo[exp_receiver_dist_and_angle][0], 'end_y':receiver_coo[exp_receiver_dist_and_angle][1]}    

        events.at[idx,'Intended_Receiver'] = Intended_dict

        # first_angle : receiver - 0.04초 볼 위치 - 0.4초 볼 위치 각도의  중 최솟값 
        ball_receiver_angle.append(np.amin(angle) * 180/math.pi)

    
    events = events[events['Intended_Receiver'] != 0]
    events['ball_receiver_angle'] = ball_receiver_angle
    return events

### Intended-receiver 평가 지표

In [112]:
# 각 방식에 대한 정확도 계산
methods = ['dist_nearest', 'angle_nearest', 'dist and angle']

# 정확도를 계산하는 함수
def calculate_accuracy(events, method):
    correct_predictions = events.apply(lambda row: row['Intended_Receiver'][method]['ID'] == row['to'], axis=1)
    accuracy = correct_predictions.mean()
    return accuracy

<h3> Metrica 1</h3>

In [113]:
df_pass_success_1 = extract_pass(all_events[all_events["game_id"] == 1], match1)
df_pass_success_1.head()

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,freeze_frame,accurate,value_label,Baseline Intended-Receiver,game_id,event_id,True Intended-receiver,no pass,Intended_Receiver,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd
0,Away,PASS,PASS,1,1,0.04,3,0.12,B19,B21,48.6,28.08,59.40,30.96,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B21', 'end_x': 59.66244, 'end...",1,1,NaN,NaN,"{'dist': {'ID': 'B21', 'end_x': 59.66244, 'end...",49.10976,27.87048,1,58.01328,30.64032,1
1,Away,PASS,PASS,1,3,0.12,17,0.68,B21,B15,59.4,30.96,62.64,15.12,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B15', 'end_x': 63.06444, 'end...",1,2,NaN,NaN,"{'dist': {'ID': 'B15', 'end_x': 63.06444, 'end...",58.01328,30.64032,1,62.09568,14.86008,1
2,Away,PASS,PASS,1,45,1.80,61,2.44,B15,B19,59.4,13.68,48.60,22.32,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B19', 'end_x': 48.33756, 'end...",1,3,NaN,NaN,"{'dist': {'ID': 'B19', 'end_x': 48.33756, 'end...",59.34168,13.63032,1,48.96828,21.89952,1
3,Away,PASS,PASS,1,77,3.08,96,3.84,B19,B21,48.6,23.04,52.92,33.84,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B21', 'end_x': 52.67376, 'end...",1,4,NaN,NaN,"{'dist': {'ID': 'B21', 'end_x': 52.67376, 'end...",48.89700,22.79952,1,52.43940,34.04376,1
4,Away,PASS,PASS,1,191,7.64,217,8.68,B21,B22,43.2,52.56,34.56,70.56,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B22', 'end_x': 33.16356, 'end...",1,5,NaN,NaN,"{'dist': {'ID': 'B22', 'end_x': 33.16356, 'end...",43.45920,52.37136,1,34.66584,69.85224,1


In [114]:
match1.head()

,frame,session,time,phase,episode,team_poss,player_poss,event_player,event_type,A11_x,A11_y,A11_vx,A11_vy,A11_speed,A11_accel,A01_x,A01_y,A01_vx,A01_vy,A01_speed,A01_accel,A02_x,A02_y,A02_vx,A02_vy,A02_speed,A02_accel,A03_x,A03_y,A03_vx,A03_vy,A03_speed,A03_accel,A04_x,A04_y,A04_vx,A04_vy,A04_speed,A04_accel,A05_x,A05_y,A05_vx,A05_vy,A05_speed,A05_accel,A06_x,A06_y,A06_vx,A06_vy,A06_speed,A06_accel,A07_x,A07_y,A07_vx,A07_vy,A07_speed,A07_accel,A08_x,A08_y,A08_vx,A08_vy,A08_speed,A08_accel,A09_x,A09_y,A09_vx,A09_vy,A09_speed,A09_accel,A10_x,A10_y,A10_vx,A10_vy,A10_speed,A10_accel,A12_x,A12_y,A12_vx,A12_vy,A12_speed,A12_accel,A13_x,A13_y,A13_vx,A13_vy,A13_speed,A13_accel,A14_x,A14_y,A14_vx,A14_vy,A14_speed,A14_accel,B25_x,B25_y,B25_vx,B25_vy,B25_speed,B25_accel,B15_x,B15_y,B15_vx,B15_vy,B15_speed,B15_accel,B16_x,B16_y,B16_vx,B16_vy,B16_speed,B16_accel,B17_x,B17_y,B17_vx,B17_vy,B17_speed,B17_accel,B18_x,B18_y,B18_vx,B18_vy,B18_speed,B18_accel,B19_x,B19_y,B19_vx,B19_vy,B19_speed,B19_accel,B20_x,B20_y,B20_vx,B20_vy,B20_speed,B20_accel,B21_x,B21_y,B21_vx,B21_vy,B21_speed,B21_accel,B22_x,B22_y,B22_vx,B22_vy,B22_speed,B22_accel,B23_x,B23_y,B23_vx,B23_vy,B23_speed,B23_accel,B24_x,B24_y,B24_vx,B24_vy,B24_speed,B24_accel,B26_x,B26_y,B26_vx,B26_vy,B26_speed,B26_accel,B27_x,B27_y,B27_vx,B27_vy,B27_speed,B27_accel,B28_x,B28_y,B28_vx,B28_vy,B28_speed,B28_accel,ball_x,ball_y,ball_vx,ball_vy,ball_speed,ball_accel
0,1,1,0.04,1,1,B,NaN,B19,PASS,0.08856,34.73136,0.432189,-0.031720,0.433351,-3.078168,35.25984,47.03184,0.053434,0.297378,0.302140,4.717885,36.39708,35.18136,0.067736,0.125874,0.142942,3.654667,33.40116,25.58088,-0.135000,0.093147,0.164016,1.440714,34.70796,15.30864,-0.130280,0.343385,0.367268,1.853775,44.38152,52.26408,-0.013972,0.299497,0.299822,2.253103,45.03384,34.44696,0.070238,0.343542,0.350649,3.718303,42.25500,23.43600,-0.177986,0.351189,0.393716,3.260189,49.01904,15.24528,-0.014098,0.292951,0.293290,2.153997,56.91276,27.34560,-0.020580,0.360787,0.361373,3.656886,59.66244,31.15368,-0.131035,0.468000,0.485998,2.014355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.74972,34.17264,-1.566000,-0.042545,1.566578,-0.442203,63.06444,14.97168,0.338350,-0.293916,0.448182,3.426367,73.07064,33.63120,0.231262,0.291776,0.372311,2.157628,72.69480,55.06272,0.207692,0.080056,0.222587,1.966273,44.04564,44.29800,0.099315,0.370070,0.383165,2.474643,49.10976,27.87048,0.128659,0.342157,0.365547,3.705662,60.43680,48.79800,-0.056643,0.199720,0.207597,2.461302,59.66244,31.15368,0.114545,0.088951,0.145027,1.763606,54.07236,67.91184,0.132545,0.085594,0.157781,2.119357,47.18844,3.60144,0.087231,0.085343,0.122035,1.640202,40.85964,19.71576,0.460133,-0.235133,0.516730,5.005156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.10976,27.87048,-2.295,-1.8,2.91668,-0.0
1,2,1,0.08,1,1,B,NaN,B19,PASS,0.10368,34.73136,0.399348,0.021566,0.399930,-1.535109,35.25984,47.03184,-0.065027,-0.223376,0.232649,6.364276,36.39708,35.18136,-0.107415,-0.113287,0.156115,3.705191,33.40116,25.58088,0.084927,-0.050450,0.098782,3.085240,34.70796,15.30864,0.054906,-0.151628,0.161263,5.440979,44.38152,52.26408,0.008157,-0.166783,0.166983,5.443840,45.03384,34.44696,-0.048487,-0.211620,0.217103,6.369215,42.25500,23.43600,0.084833,-0.207252,0.223942,6.991242,49.01904,15.24528,-0.002366,-0.167001,0.167018,5.671869,56.91276,27.34560,0.010422,-0.236958,0.237187,7.789423,59.66244,31.15368,0.019750,-0.207818,0.208755,6.897011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.73352,34.17264,-1.554709,0.028145,1.554964,-0.972968,63.06444,14.97168,-0.165097,0.086324,0.186303,4.474152,73.07064,33.63120,-0.107131,-0.125471,0.164985,5.182223,72.69480,55.06272,-0.110341,-0.083832,0.138575,4.665058,44.04564,44.29800,-0.069747,-0.170736,0.184432,6.169288,49.10976,27.87048,-0.115364,-0.213155,0.242371,7.568237,60.43680,48.79800,0.050979,-0.179748,0.186838,6.498264,59.66244,31.15368,-0.103091,-0.080056,0.130525,4.528301,54.072

In [115]:
df_pass_success_1.head()

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,freeze_frame,accurate,value_label,Baseline Intended-Receiver,game_id,event_id,True Intended-receiver,no pass,Intended_Receiver,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd
0,Away,PASS,PASS,1,1,0.04,3,0.12,B19,B21,48.6,28.08,59.40,30.96,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B21', 'end_x': 59.66244, 'end...",1,1,NaN,NaN,"{'dist': {'ID': 'B21', 'end_x': 59.66244, 'end...",49.10976,27.87048,1,58.01328,30.64032,1
1,Away,PASS,PASS,1,3,0.12,17,0.68,B21,B15,59.4,30.96,62.64,15.12,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B15', 'end_x': 63.06444, 'end...",1,2,NaN,NaN,"{'dist': {'ID': 'B15', 'end_x': 63.06444, 'end...",58.01328,30.64032,1,62.09568,14.86008,1
2,Away,PASS,PASS,1,45,1.80,61,2.44,B15,B19,59.4,13.68,48.60,22.32,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B19', 'end_x': 48.33756, 'end...",1,3,NaN,NaN,"{'dist': {'ID': 'B19', 'end_x': 48.33756, 'end...",59.34168,13.63032,1,48.96828,21.89952,1
3,Away,PASS,PASS,1,77,3.08,96,3.84,B19,B21,48.6,23.04,52.92,33.84,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B21', 'end_x': 52.67376, 'end...",1,4,NaN,NaN,"{'dist': {'ID': 'B21', 'end_x': 52.67376, 'end...",48.89700,22.79952,1,52.43940,34.04376,1
4,Away,PASS,PASS,1,191,7.64,217,8.68,B21,B22,43.2,52.56,34.56,70.56,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B22', 'end_x': 33.16356, 'end...",1,5,NaN,NaN,"{'dist': {'ID': 'B22', 'end_x': 33.16356, 'end...",43.45920,52.37136,1,34.66584,69.85224,1


In [116]:
df_pass_success_1 = get_nearest_receiver(df_pass_success_1, match1)
accuracies= {method: calculate_accuracy(df_pass_success_1, method) for method in methods}
accuracies

C:\Users\M207\AppData\Local\Temp\ipykernel_25168\178272040.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['ball_receiver_angle'] = ball_receiver_angle


{'dist_nearest': 0.28388746803069054,
 'angle_nearest': 0.7161125319693095,
 'dist and angle': 0.7979539641943734}

<h3> Metrica 2</h3>

In [93]:
df_pass_success_2 = extract_pass(all_events[all_events["game_id"] == 2], match2)
df_pass_success_2.head()

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,freeze_frame,accurate,value_label,Baseline Intended-Receiver,game_id,event_id,True Intended-receiver,no pass,Intended_Receiver,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd
0,Away,PASS,PASS,1,51,2.04,87,3.48,B23,B20,54.00,36.00,43.20,36.72,1,0,0,2,Pass,"{'A01': {'teammate': False, 'actor': False, 'b...",1.0,0,"{'dist': {'ID': 'B20', 'end_x': 64.75464, 'end...",2,1746,NaN,NaN,"{'dist': {'ID': 'B20', 'end_x': 64.75464, 'end...",54.23328,36.27072,1,43.83504,36.59544,1
1,Away,PASS,PASS,1,146,5.84,186,7.44,B20,B18,46.44,36.00,47.52,15.84,1,0,0,2,Pass,"{'A01': {'teammate': False, 'actor': False, 'b...",1.0,0,"{'dist': {'ID': 'B18', 'end_x': 63.23184, 'end...",2,1747,NaN,NaN,"{'dist': {'ID': 'B18', 'end_x': 63.23184, 'end...",45.88488,36.14760,1,47.85372,16.16544,1
2,Away,PASS,PASS,1,248,9.92,283,11.32,B18,B17,50.76,13.68,33.48,20.16,1,0,0,2,Pass,"{'A01': {'teammate': False, 'actor': False, 'b...",1.0,0,"{'dist': {'ID': 'B17', 'end_x': 73.34388, 'end...",2,1748,NaN,NaN,"{'dist': {'ID': 'B17', 'end_x': 73.34388, 'end...",50.23620,13.48488,1,33.75216,20.10600,1
3,Away,PASS,PASS,1,316,12.64,346,13.84,B17,B16,31.32,23.04,28.08,41.76,1,0,0,2,Pass,"{'A01': {'teammate': False, 'actor': False, 'b...",1.0,0,"{'dist': {'ID': 'B16', 'end_x': 77.99004, 'end...",2,1749,NaN,NaN,"{'dist': {'ID': 'B16', 'end_x': 77.99004, 'end...",31.75632,22.69584,1,28.44720,41.01552,1
4,Away,PASS,PASS,1,395,15.80,423,16.92,B16,B15,28.08,47.52,29.16,65.52,1,0,0,2,Pass,"{'A01': {'teammate': False, 'actor': False, 'b...",1.0,0,"{'dist': {'ID': 'B15', 'end_x': 76.3614, 'end_...",2,1750,NaN,NaN,"{'dist': {'ID': 'B15', 'end_x': 76.3614, 'end_...",27.77436,47.25504,1,28.99368,65.00520,1


In [117]:
df_pass_success_2 = get_nearest_receiver(df_pass_success_2, match2)
accuracies = {method: calculate_accuracy(df_pass_success_2, method) for method in methods}
accuracies

C:\Users\M207\AppData\Local\Temp\ipykernel_25168\178272040.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['ball_receiver_angle'] = ball_receiver_angle


{'dist_nearest': 0.3357817418677859,
 'angle_nearest': 0.7481636935991606,
 'dist and angle': 0.8132214060860441}

<h3> Metrica 3</h3>

In [118]:
df_pass_success_3 = all_events[all_events["game_id"] == 3]
df_pass_success_3.replace({'team': {'Away': 'Home', 'Home': 'Away'}}, inplace=True)

df_pass_success_3 = extract_pass(df_pass_success_3, match3)
df_pass_success_3.head()

C:\Users\M207\AppData\Local\Temp\ipykernel_25168\784285754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pass_success_3.replace({'team': {'Away': 'Home', 'Home': 'Away'}}, inplace=True)


,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,freeze_frame,accurate,value_label,Baseline Intended-Receiver,game_id,event_id,True Intended-receiver,no pass,Intended_Receiver,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd
0,Home,PASS,PASS,1,361,14.44,377,15.08,A10,A07,54.13500,35.08200,53.85312,35.06760,1,0,0,2,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'A07', 'end_x': 54.13716, 'end...",3,3681,NaN,NaN,"{'dist': {'ID': 'A07', 'end_x': 54.13716, 'end...",54.13500,35.08200,1,53.86392,35.06832,1
1,Home,PASS,PASS,1,384,15.36,426,17.04,A07,A08,53.67600,34.92000,68.44284,45.68328,1,0,0,2,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'A08', 'end_x': 66.78936, 'end...",3,3683,NaN,NaN,"{'dist': {'ID': 'A08', 'end_x': 66.78936, 'end...",53.70192,34.94304,1,68.12100,45.44928,1
2,Home,PASS,PASS,1,465,18.60,507,20.28,A08,A02,72.34488,42.98904,87.05016,28.67112,1,0,0,2,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'A02', 'end_x': 84.64716, 'end...",3,3685,NaN,NaN,"{'dist': {'ID': 'A02', 'end_x': 84.64716, 'end...",72.21960,43.09560,1,86.88600,28.83096,1
3,Home,PASS,PASS,1,530,21.20,580,23.20,A02,A03,87.40332,30.90384,86.29848,58.69584,1,0,0,2,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'A03', 'end_x': 85.1742, 'end_...",3,3687,NaN,NaN,"{'dist': {'ID': 'A03', 'end_x': 85.1742, 'end_...",87.36120,30.85992,1,86.30388,58.57056,1
4,Home,PASS,PASS,1,598,23.92,628,25.12,A03,A04,86.13648,59.03856,73.54908,70.60248,1,0,0,2,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'A04', 'end_x': 73.46808, 'end...",3,3689,NaN,NaN,"{'dist': {'ID': 'A04', 'end_x': 73.46808, 'end...",86.14188,59.02272,1,73.72728,70.43904,1


In [119]:
df_pass_success_3 = get_nearest_receiver(df_pass_success_3, match3)
accuracies = {method: calculate_accuracy(df_pass_success_3, method) for method in methods}
accuracies

C:\Users\M207\AppData\Local\Temp\ipykernel_25168\178272040.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['ball_receiver_angle'] = ball_receiver_angle


{'dist_nearest': 0.3092046470062556,
 'angle_nearest': 0.7158176943699732,
 'dist and angle': 0.8114387846291331}

### Success_all_match.csv



In [120]:
success_all_match = pd.concat([df_pass_success_1, df_pass_success_2, df_pass_success_3])
success_all_match.head()

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,freeze_frame,accurate,value_label,Baseline Intended-Receiver,game_id,event_id,True Intended-receiver,no pass,Intended_Receiver,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd,ball_receiver_angle
1,Away,PASS,PASS,1,3,0.12,17,0.68,B21,B15,59.40,30.96,62.64,15.12,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B15', 'end_x': 63.06444, 'end...",1,2,NaN,NaN,"{'dist_nearest': {'ID': 'B19', 'end_x': 49.109...",58.01328,30.64032,1,62.09568,14.86008,1,0.157514
2,Away,PASS,PASS,1,45,1.80,61,2.44,B15,B19,59.40,13.68,48.60,22.32,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B19', 'end_x': 48.33756, 'end...",1,3,NaN,NaN,"{'dist_nearest': {'ID': 'B19', 'end_x': 48.337...",59.34168,13.63032,1,48.96828,21.89952,1,1.394635
3,Away,PASS,PASS,1,77,3.08,96,3.84,B19,B21,48.60,23.04,52.92,33.84,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B21', 'end_x': 52.67376, 'end...",1,4,NaN,NaN,"{'dist_nearest': {'ID': 'B21', 'end_x': 52.673...",48.89700,22.79952,1,52.43940,34.04376,1,5.611194
4,Away,PASS,PASS,1,191,7.64,217,8.68,B21,B22,43.20,52.56,34.56,70.56,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B22', 'end_x': 33.16356, 'end...",1,5,NaN,NaN,"{'dist_nearest': {'ID': 'B20', 'end_x': 51.938...",43.45920,52.37136,1,34.66584,69.85224,1,2.323705
5,Away,PASS,PASS,1,279,11.16,303,12.12,B22,B17,42.12,69.12,52.92,70.56,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",1.0,0,"{'dist': {'ID': 'B17', 'end_x': 51.54516, 'end...",1,6,NaN,NaN,"{'dist_nearest': {'ID': 'B17', 'end_x': 51.545...",41.56164,69.27552,1,52.98156,70.24536,1,5.453404


In [121]:
accuracies = {method: calculate_accuracy(success_all_match, method) for method in methods}
accuracies

{'dist_nearest': 0.31114225648213034,
 'angle_nearest': 0.7266993693062369,
 'dist and angle': 0.8083391730903995}

### Unsuccess_all_match.csv

In [122]:
match = [match1, match2, match3]
unsuccess_match = []

for i in range(1, 4):
    df_pass_unsuccess = all_events[all_events["game_id"] == i]
    if i == 3:
        df_pass_unsuccess.replace({'team': {'Away': 'Home', 'Home': 'Away'}}, inplace=True)
    df_pass_unsuccess = extract_pass(df_pass_unsuccess, match[i-1], 0)
    unsuccess_match.append(df_pass_unsuccess)

C:\Users\M207\AppData\Local\Temp\ipykernel_25168\3932208390.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pass_unsuccess.replace({'team': {'Away': 'Home', 'Home': 'Away'}}, inplace=True)


In [123]:
unsuccess_all_match = pd.concat(unsuccess_match)
# unsuccess_all_match.to_csv('../metrica-data/EPV-data/all-match-unsuccess.csv')

unsuccess_all_match.head()

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,freeze_frame,accurate,value_label,Baseline Intended-Receiver,game_id,event_id,True Intended-receiver,no pass,Intended_Receiver,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd
6,Away,BALL LOST,INTERCEPTION,1,346,13.84,380,15.20,B17,NaN,55.08,69.84,29.16,54.00,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'B18', 'end_x': 34.398, 'end_y...",1,7,B24,NaN,"{'dist': {'ID': 'B18', 'end_x': 34.398, 'end_y...",54.98172,69.67944,1,29.59956,55.85472,1
7,Home,BALL LOST,INTERCEPTION,1,378,15.12,452,18.08,A02,NaN,29.16,56.16,63.72,46.08,1,0,0,1,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'A10', 'end_x': 51.99768, 'end...",1,9,A10,NaN,"{'dist': {'ID': 'A10', 'end_x': 51.99768, 'end...",28.94400,56.04264,1,63.31068,46.19736,1
10,Home,BALL LOST,INTERCEPTION,1,572,22.88,616,24.64,A10,NaN,54.00,46.80,72.36,31.68,1,0,0,1,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'A08', 'end_x': 60.2208, 'end_...",1,17,A08,NaN,"{'dist': {'ID': 'A08', 'end_x': 60.2208, 'end_...",54.22572,47.08224,1,72.33948,31.69800,1
15,Away,BALL LOST,INTERCEPTION,1,1110,44.40,1134,45.36,B17,NaN,45.36,56.88,33.48,60.48,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'B21', 'end_x': 29.4732, 'end_...",1,23,B21,NaN,"{'dist': {'ID': 'B21', 'end_x': 29.4732, 'end_...",45.40860,56.76984,1,34.44228,63.78912,1
19,Home,BALL LOST,INTERCEPTION,1,1370,54.80,1375,55.00,A08,NaN,92.88,18.72,95.04,20.16,1,0,0,1,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'A09', 'end_x': 95.44284, 'end...",1,28,A09,NaN,"{'dist': {'ID': 'A09', 'end_x': 95.44284, 'end...",92.51280,19.03032,1,94.72680,19.98720,1


### Unsuccess Intended Receiver Baseline 확인

In [124]:
# 각 방식에 대한 정확도 계산
methods = ['dist_nearest', 'angle_nearest', 'dist and angle']

# 정확도를 계산하는 함수
def calculate_accuracy(events, method):
    correct_predictions = events.apply(lambda row: row['Intended_Receiver'][method]['ID'] == row['Intended_labelling'], axis=1)
    accuracy = correct_predictions.mean()
    return accuracy

In [125]:
un_label = pd.read_csv('../metrica-data/EPV-data/unsuccess_passes.csv', index_col=0)
un_label = un_label[un_label['Intended_labelling'].isna() == False]
un_label.head()

,subtype,game_id,event_id,from,Intended_labelling,no pass
type,,,,,,
BALL LOST,INTERCEPTION,1,7,B17,24.0,NaN
BALL LOST,INTERCEPTION,1,9,A02,10.0,NaN
BALL LOST,INTERCEPTION,1,17,A10,8.0,NaN
BALL LOST,INTERCEPTION,1,23,B17,21.0,NaN
BALL LOST,INTERCEPTION,1,28,A08,9.0,NaN


In [126]:
# to/from 양식과 똑같이 Intended_labelling 칼럼을 바꿔줌.
m_list = []
for idx, action in un_label.iterrows():
    num = str(int(action['Intended_labelling']))
    if len(num) == 1:
        m = action['from'][0] + '0' + num
    else:
        m = action['from'][0] + num
    m_list.append(m)
un_label['Intended_labelling'] = m_list

un_label.head()

,subtype,game_id,event_id,from,Intended_labelling,no pass
type,,,,,,
BALL LOST,INTERCEPTION,1,7,B17,B24,NaN
BALL LOST,INTERCEPTION,1,9,A02,A10,NaN
BALL LOST,INTERCEPTION,1,17,A10,A08,NaN
BALL LOST,INTERCEPTION,1,23,B17,B21,NaN
BALL LOST,INTERCEPTION,1,28,A08,A09,NaN


In [127]:
len(unsuccess_all_match)

411

#### Metrica 1

In [128]:
unsuccess_metrica_1 = unsuccess_all_match[unsuccess_all_match['game_id'] == 1].copy()
un_label1 = un_label[un_label['game_id'] == 1].copy()

In [129]:
# join
df_un_1 = pd.merge(unsuccess_metrica_1, un_label1[['event_id', 'Intended_labelling']], how='inner', on='event_id')
# labelling 된 것들만
df_pass_unsuccess_1 = df_un_1[df_un_1['Intended_labelling'].isna() == False].copy()

df_pass_unsuccess_1.head()

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,freeze_frame,accurate,value_label,Baseline Intended-Receiver,game_id,event_id,True Intended-receiver,no pass,Intended_Receiver,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd,Intended_labelling
0,Away,BALL LOST,INTERCEPTION,1,346,13.84,380,15.20,B17,NaN,55.08,69.84,29.16,54.00,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'B18', 'end_x': 34.398, 'end_y...",1,7,B24,NaN,"{'dist': {'ID': 'B18', 'end_x': 34.398, 'end_y...",54.98172,69.67944,1,29.59956,55.85472,1,B24
1,Home,BALL LOST,INTERCEPTION,1,378,15.12,452,18.08,A02,NaN,29.16,56.16,63.72,46.08,1,0,0,1,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'A10', 'end_x': 51.99768, 'end...",1,9,A10,NaN,"{'dist': {'ID': 'A10', 'end_x': 51.99768, 'end...",28.94400,56.04264,1,63.31068,46.19736,1,A10
2,Home,BALL LOST,INTERCEPTION,1,572,22.88,616,24.64,A10,NaN,54.00,46.80,72.36,31.68,1,0,0,1,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'A08', 'end_x': 60.2208, 'end_...",1,17,A08,NaN,"{'dist': {'ID': 'A08', 'end_x': 60.2208, 'end_...",54.22572,47.08224,1,72.33948,31.69800,1,A08
3,Away,BALL LOST,INTERCEPTION,1,1110,44.40,1134,45.36,B17,NaN,45.36,56.88,33.48,60.48,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'B21', 'end_x': 29.4732, 'end_...",1,23,B21,NaN,"{'dist': {'ID': 'B21', 'end_x': 29.4732, 'end_...",45.40860,56.76984,1,34.44228,63.78912,1,B21
4,Home,BALL LOST,INTERCEPTION,1,1370,54.80,1375,55.00,A08,NaN,92.88,18.72,95.04,20.16,1,0,0,1,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'A09', 'end_x': 95.44284, 'end...",1,28,A09,NaN,"{'dist': {'ID': 'A09', 'end_x': 95.44284, 'end...",92.51280,19.03032,1,94.72680,19.98720,1,A09


In [130]:
df_pass_unsuccess_1 = get_nearest_receiver(df_pass_unsuccess_1, match1)
accuracies = {method: calculate_accuracy(df_pass_unsuccess_1, method) for method in methods}
accuracies

C:\Users\M207\AppData\Local\Temp\ipykernel_25168\178272040.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['ball_receiver_angle'] = ball_receiver_angle


{'dist_nearest': 0.13740458015267176,
 'angle_nearest': 0.5648854961832062,
 'dist and angle': 0.5419847328244275}

#### Metrica 2

In [131]:
unsuccess_metrica_2 = unsuccess_all_match[unsuccess_all_match['game_id'] == 2].copy()
un_label2 = un_label[un_label['game_id'] == 2].copy()

In [132]:
# join
df_pass_unsuccess_2 = pd.merge(unsuccess_metrica_2, un_label2[['event_id', 'Intended_labelling']], how='inner', on='event_id')
df_pass_unsuccess_2.head()

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,freeze_frame,accurate,value_label,Baseline Intended-Receiver,game_id,event_id,True Intended-receiver,no pass,Intended_Receiver,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd,Intended_labelling
0,Away,BALL LOST,INTERCEPTION,1,451,18.04,504,20.16,B15,NaN,28.08,66.24,69.12,66.96,1,0,0,2,Pass,"{'A01': {'teammate': False, 'actor': False, 'b...",0.0,0,"{'dist': {'ID': 'B23', 'end_x': 44.75196, 'end...",2,1751,B23,NaN,"{'dist': {'ID': 'B23', 'end_x': 44.75196, 'end...",28.14696,66.14856,1,67.18464,66.46680,1,B23
1,Away,BALL LOST,INTERCEPTION,1,1016,40.64,1087,43.48,B16,NaN,16.20,64.80,55.08,50.40,1,0,0,2,Pass,"{'A01': {'teammate': False, 'actor': False, 'b...",0.0,0,"{'dist': {'ID': 'B24', 'end_x': 48.48984, 'end...",2,1762,B24,NaN,"{'dist': {'ID': 'B24', 'end_x': 48.48984, 'end...",16.11576,64.70784,1,53.59716,53.39592,1,B24
2,Away,BALL LOST,INTERCEPTION,1,1392,55.68,1429,57.16,B24,NaN,77.76,2.88,93.96,9.36,1,0,0,2,Pass,"{'A01': {'teammate': False, 'actor': False, 'b...",0.0,0,"{'dist': {'ID': 'B22', 'end_x': 21.22307999999...",2,1771,B22,NaN,"{'dist': {'ID': 'B22', 'end_x': 21.22307999999...",77.67684,2.85408,1,93.67056,9.24120,1,B22
3,Home,BALL LOST,INTERCEPTION,1,1578,63.12,1611,64.44,A09,NaN,63.72,10.08,56.16,20.88,1,0,0,1,Pass,"{'A01': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'A10', 'end_x': 52.21584, 'end...",2,1777,A10,NaN,"{'dist': {'ID': 'A10', 'end_x': 52.21584, 'end...",63.75132,10.32552,1,56.18808,20.83968,1,A10
4,Away,BALL LOST,INTERCEPTION,1,1843,73.72,1876,75.04,B15,NaN,60.48,60.48,74.52,51.84,1,0,0,2,Pass,"{'A01': {'teammate': False, 'actor': False, 'b...",0.0,0,"{'dist': {'ID': 'B21', 'end_x': 41.21928, 'end...",2,1786,B24,NaN,"{'dist': {'ID': 'B21', 'end_x': 41.21928, 'end...",60.79104,59.80536,1,72.87732,51.47928,1,B24


In [133]:
df_pass_unsuccess_2 = get_nearest_receiver(df_pass_unsuccess_2, match2)
accuracies = {method: calculate_accuracy(df_pass_unsuccess_2, method) for method in methods}
accuracies

C:\Users\M207\AppData\Local\Temp\ipykernel_25168\178272040.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['ball_receiver_angle'] = ball_receiver_angle


{'dist_nearest': 0.15454545454545454,
 'angle_nearest': 0.5363636363636364,
 'dist and angle': 0.4818181818181818}

#### Metrica 3

In [134]:
unsuccess_metrica_3 = unsuccess_all_match[unsuccess_all_match['game_id'] == 3].copy()
un_label3 = un_label[un_label['game_id'] == 3].copy()

In [135]:
# join
df_pass_unsuccess_3 = pd.merge(unsuccess_metrica_3, un_label3[['event_id', 'Intended_labelling']], how='inner', on='event_id')
df_pass_unsuccess_3.head()

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,freeze_frame,accurate,value_label,Baseline Intended-Receiver,game_id,event_id,True Intended-receiver,no pass,Intended_Receiver,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd,Intended_labelling
0,Away,BALL LOST,INTERCEPTION,1,2364,94.56,2493,99.72,B18,NaN,47.01888,70.83000,101.52,69.12,1,0,0,1,Pass,"{'B28': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'B26', 'end_x': 67.63824, 'end...",3,3745,B26,NaN,"{'dist': {'ID': 'B26', 'end_x': 67.63824, 'end...",46.93032,70.84296,1,99.13752,68.11416,1,B26
1,Home,BALL LOST,INTERCEPTION,1,2923,116.92,2950,118.00,A04,NaN,96.34248,70.81128,92.88,67.68,1,0,0,2,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'A07', 'end_x': 88.65936, 'end...",3,3757,A07,NaN,"{'dist': {'ID': 'A07', 'end_x': 88.65936, 'end...",96.34896,70.83720,2,92.95236,67.74552,2,A07
2,Home,BALL LOST,INTERCEPTION,1,3625,145.00,3626,145.04,A06,NaN,61.66800,4.84992,58.32,2.88,1,0,0,2,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'A05', 'end_x': 56.92248, 'end...",3,3783,A05,NaN,"{'dist': {'ID': 'A05', 'end_x': 56.92248, 'end...",61.71984,4.82328,2,58.52952,3.00312,2,A05
3,Away,BALL LOST,INTERCEPTION,1,4385,175.40,4408,176.32,B20,NaN,13.34232,6.89976,41.04,16.56,1,0,0,1,Pass,"{'B28': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'B24', 'end_x': 30.63312, 'end...",3,3801,B25,NaN,"{'dist': {'ID': 'B24', 'end_x': 30.63312, 'end...",13.29372,6.90480,3,37.11636,13.47048,3,B25
4,Home,BALL LOST,INTERCEPTION,1,6449,257.96,6487,259.48,A07,NaN,-1.56168,0.19080,7.56,34.56,1,0,0,2,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'A02', 'end_x': 9.39384, 'end_...",3,3815,A10,NaN,"{'dist': {'ID': 'A02', 'end_x': 9.39384, 'end_...",-1.56168,0.19080,5,7.52436,34.42464,5,A10


In [136]:
df_pass_unsuccess_3 = get_nearest_receiver(df_pass_unsuccess_3, match3)
accuracies = {method: calculate_accuracy(df_pass_unsuccess_3, method) for method in methods}
accuracies

C:\Users\M207\AppData\Local\Temp\ipykernel_25168\178272040.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['ball_receiver_angle'] = ball_receiver_angle


{'dist_nearest': 0.2,
 'angle_nearest': 0.49411764705882355,
 'dist and angle': 0.5176470588235295}

### Unsuccess-match.csv

In [137]:
unsuccess_all_match = pd.concat([df_pass_unsuccess_1, df_pass_unsuccess_2, df_pass_unsuccess_3])
unsuccess_all_match['to'] = unsuccess_all_match['Intended_labelling']
unsuccess_all_match.head()

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,freeze_frame,accurate,value_label,Baseline Intended-Receiver,game_id,event_id,True Intended-receiver,no pass,Intended_Receiver,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd,Intended_labelling,ball_receiver_angle
0,Away,BALL LOST,INTERCEPTION,1,346,13.84,380,15.20,B17,B24,55.08,69.84,29.16,54.00,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'B18', 'end_x': 34.398, 'end_y...",1,7,B24,NaN,"{'dist_nearest': {'ID': 'B22', 'end_x': 46.564...",54.98172,69.67944,1,29.59956,55.85472,1,B24,5.669100
1,Home,BALL LOST,INTERCEPTION,1,378,15.12,452,18.08,A02,A10,29.16,56.16,63.72,46.08,1,0,0,1,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'A10', 'end_x': 51.99768, 'end...",1,9,A10,NaN,"{'dist_nearest': {'ID': 'A03', 'end_x': 24.845...",28.94400,56.04264,1,63.31068,46.19736,1,A10,3.701397
2,Home,BALL LOST,INTERCEPTION,1,572,22.88,616,24.64,A10,A08,54.00,46.80,72.36,31.68,1,0,0,1,Pass,"{'A11': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'A08', 'end_x': 60.2208, 'end_...",1,17,A08,NaN,"{'dist_nearest': {'ID': 'A09', 'end_x': 56.728...",54.22572,47.08224,1,72.33948,31.69800,1,A08,32.044545
3,Away,BALL LOST,INTERCEPTION,1,1110,44.40,1134,45.36,B17,B21,45.36,56.88,33.48,60.48,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'B21', 'end_x': 29.4732, 'end_...",1,23,B21,NaN,"{'dist_nearest': {'ID': 'B20', 'end_x': 40.176...",45.40860,56.76984,1,34.44228,63.78912,1,B21,1.877354
5,Away,BALL LOST,INTERCEPTION,1,4489,179.56,4518,180.72,B19,B24,58.32,62.64,51.84,61.20,1,0,0,2,Pass,"{'B25': {'teammate': True, 'actor': False, 'ba...",0.0,0,"{'dist': {'ID': 'B20', 'end_x': 62.1594, 'end_...",1,46,B24,NaN,"{'dist_nearest': {'ID': 'B20', 'end_x': 62.159...",58.18068,62.60616,4,54.15228,62.88768,4,B24,2.791807


In [138]:
accuracies = {method: calculate_accuracy(unsuccess_all_match, method) for method in methods}
accuracies

{'dist_nearest': 0.15950920245398773,
 'angle_nearest': 0.5368098159509203,
 'dist and angle': 0.5153374233128835}

### 전체 성능

In [139]:
# 각 방식에 대한 정확도 계산
methods = ['dist_nearest', 'angle_nearest', 'dist and angle']

# 정확도를 계산하는 함수
def calculate_accuracy(events, method):
    correct_predictions = events.apply(lambda row: row['Intended_Receiver'][method]['ID'] == row['to'], axis=1)
    accuracy = correct_predictions.mean()
    return accuracy

In [140]:
all_pass = pd.concat([success_all_match, unsuccess_all_match])
accuracies = {method: calculate_accuracy(all_pass, method) for method in methods}
accuracies

{'dist_nearest': 0.29559748427672955,
 'angle_nearest': 0.7072327044025157,
 'dist and angle': 0.7783018867924528}

### 패스 데이터 확인

In [141]:
# 총 패스 데이터 = 성공한 패스(2882) + 라벨링된 실패 패스(367) + 라벨링되지 못한 실패 패스(43)
print(f'총 패스 데이터 수: {len(all_events[(all_events["eventName"] == "Pass") & (all_events["start_frame"] < all_events["end_frame"])])}')
print(f'성공 패스 데이터 수: {len(success_all_match)}')
print(f'실패 패스 데이터 수: {len(unsuccess_all_match)}')

총 패스 데이터 수: 3293
성공 패스 데이터 수: 2854
실패 패스 데이터 수: 326
